## Build Audio Vectors
Now that the labels have been extracted, we'll use the compiled csv (df_iemocap.csv) to split the original wav files into multiple frames

In [1]:
# Try for one file first
import librosa
import os
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as ms
from tqdm import tqdm
import pickle

import IPython.display
import librosa.display
# ms.use('seaborn-muted')
%matplotlib inline

In [2]:
file_path = '/Users/sensenc/Downloads/IEMOCAP_full_release/Session1/dialog/wav/Ses01F_impro01.wav'

y, sr = librosa.load(file_path, sr=44100)
y, sr

(array([ 0.4256729 ,  0.4850127 ,  0.3723262 , ..., -0.31710678,
        -0.16457509,  0.        ], dtype=float32),
 44100)

## Loop through all the files

In [3]:
import pandas as pd
import math

labels_df = pd.read_csv('../data/preprocessing/df_iemocap.csv')
iemocap_dir = '/Users/sensenc/Downloads/IEMOCAP_full_release/'

The following cells take some time until completely executed

In [4]:
sr = 44100
audio_vectors = {}
for sess in range(1, 6):  # using one session due to memory constraint, can replace [5] with range(1, 6)
    wav_file_path = '{}Session{}/dialog/wav/'.format(iemocap_dir, sess)
    orig_wav_files = os.listdir(wav_file_path)
    for orig_wav_file in tqdm(orig_wav_files):
        try:
            orig_wav_vector, _sr = librosa.load(wav_file_path + orig_wav_file, sr=sr)
            orig_wav_file, file_format = orig_wav_file.split('.')
            for index, row in labels_df[labels_df['wav_file'].str.contains(orig_wav_file)].iterrows():
                start_time, end_time, truncated_wav_file_name, emotion, val, act, dom = row['start_time'], row['end_time'], row['wav_file'], row['emotion'], row['val'], row['act'], row['dom']
                start_frame = math.floor(start_time * sr)
                end_frame = math.floor(end_time * sr)
                truncated_wav_vector = orig_wav_vector[start_frame:end_frame + 1]
                audio_vectors[truncated_wav_file_name] = truncated_wav_vector
        except:
            print('An exception occured for {}'.format(orig_wav_file))
    with open('../data/preprocessing/audio_vectors_{}.pkl'.format(sess), 'wb') as f:
        pickle.dump(audio_vectors, f)

 97%|█████████▋| 30/31 [01:26<00:02,  2.43s/it]/Users/sensenc/anaconda3/envs/sound_detection_project/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
100%|██████████| 31/31 [01:26<00:00,  2.80s/it]


An exception occured for Ses02F_script01_1.pk


100%|██████████| 31/31 [01:47<00:00,  3.46s/it]
